In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import psycopg2
import glob

from datetime import date


In [2]:
# Function to get current working directory
def current_path():
    print("Current Directory:")
    print(os.getcwd())
    print()
    
# Function to get new directory 
def new_path():
    print("New Directory:")
    print(os.getcwd())
    print()
 
 
# Driver's code
# Printing CWD before
current_path()
 
# Changing the CWD
os.chdir('../../../Work Order - Data/WO_Folders/WO_2022-2023')
 
# Printing CWD after
new_path()

#Itererates and gets the latest file
cwd = os.getcwd()
files = [x for x in os.listdir(cwd) if x.endswith(".csv")]
newest_file = max(files, key = os.path.getctime)
print("Latest CSV Received: ", newest_file)

#Ensuring that the files is chosen:
os.path.exists(newest_file)

Current Directory:
C:\Users\E178162\Documents\Work Orders\DB\Jupyter Notebooks\Data Cleanse

New Directory:
C:\Users\E178162\Documents\Work Orders\Work Order - Data\WO_Folders\WO_2022-2023

Latest CSV Received:  Work_Order_(02_27_2023_09_51_28_AM).csv


True

In [3]:
dtype_dict = {'Work Order #': str}

df = pd.read_csv(newest_file, dtype=dtype_dict)

In [4]:
df['Date Open'] = pd.to_datetime(df['Created On']).dt.date
df['Date Closed'] = pd.to_datetime(df['Completed']).dt.date #Creates new column and sets data type from DateTime -> Date

df['Date Difference'] = date.today() - df['Date Open'] #Takes the difference between today's date from Date Open
df['Date Difference'] = df['Date Difference'].dt.days #Converts DateTime Type to Integer

df['Duration'] = df['Date Closed'] - df['Date Open'] #Takes the diffrence between Date Closed from Date Open
df['Duration'] = df['Duration'].dt.days #Converts DateTime Type to Integer
df['Duration'] = df['Duration'].abs() #Gets the Absolute Value of Column Values

df['Site_ID'] = df['Source Site'].str[-3:] #Takes the last 3 characters froms string in this case, we get extract the Site Code

df['School Year'] = '2022 - 2023' #Create new column of School Year

df['Duration'] = df['Duration'].fillna(df['Date Difference']) #Fill null values from Duration column with values from Date Difference

df['Row Count'] = 1 #Create new column of Row Count

df = df.drop(columns=['Area/Room #', 'Originator', 'Originator First Name', 'Work requested', 'Cause', 'Action Taken', 'Comments' ,'Longitude', 'Latitude', 'PM Title', 'Completed','Created On', 'Originated']) #Drops/Removes the columns from table

df = df[df['Site_ID'].str.contains("ZZ-|032|941") == False] #Remove any string or cell that contains those characters

df = df.drop_duplicates(subset='Work Order #', keep='first') #Remove any duplicates

# Delete rows where work order is present but work category is null
df = df.dropna(subset=['Work Category'])

df = df.rename(columns={"Work Order #": "Work Order ID"})

columns = ['Work Order ID', 'WO Status', 'Priority', 'Work Category', 'Problem',
              'Source Site', 'Site_ID' , 'Source Location', 'Source User', 
              'Date Open', 'Date Closed', 'Date Difference', 'Duration',
              'School Year','Work Type', 'Labor Hrs', 'Part Cost', 'Labor Cost',
              'Total Hrs', 'Grand Total', 'Row Count'] 

df = df[columns]

df['Site_ID'] = df['Site_ID'].astype(int)

df["Source User"].fillna("Pending Assignment", inplace=True)

work_order_report = df

work_order_report.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36000 entries, 0 to 36009
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Work Order ID    36000 non-null  object 
 1   WO Status        36000 non-null  object 
 2   Priority         34306 non-null  object 
 3   Work Category    36000 non-null  object 
 4   Problem          36000 non-null  object 
 5   Source Site      36000 non-null  object 
 6   Site_ID          36000 non-null  int32  
 7   Source Location  34814 non-null  object 
 8   Source User      36000 non-null  object 
 9   Date Open        36000 non-null  object 
 10  Date Closed      31491 non-null  object 
 11  Date Difference  36000 non-null  int64  
 12  Duration         36000 non-null  float64
 13  School Year      36000 non-null  object 
 14  Work Type        20953 non-null  object 
 15  Labor Hrs        36000 non-null  float64
 16  Part Cost        36000 non-null  float64
 17  Labor Cost  

In [5]:
null_category_wos = df.loc[df['Work Category'].isnull(), 'Work Order ID']
print(null_category_wos)

Series([], Name: Work Order ID, dtype: object)


In [6]:
def generate_work_order_report(df, groupby_col):
    # group the data by department
    grouped = df.groupby(df[groupby_col].fillna('Pending Assignment'))

    # calculate the total work orders and work orders completed
    total_wos = grouped['Work Order ID'].nunique().astype(int)
    wos_completed = grouped['Date Closed'].count().astype(int)

    # calculate the work orders in progress
    in_progress = ((df['Date Closed'].isnull()) & (df['Duration'] > 10)).groupby(df[groupby_col]).sum().astype(int)

    # calculate the new request work orders
    new_request = ((df['Date Closed'].isnull()) & (df['Duration'] >= 0) & (df['Duration'] <= 10)).groupby(df[groupby_col]).sum().astype(int)

    # calculate the average work order duration
    avg_duration = grouped['Duration'].mean().round(1)

    # get the completion rate for each department
    wo_completion_rate = (wos_completed / total_wos).round(4)

    # convert the completion rate to a percentage
    wo_completion_rate_percent = (wo_completion_rate * 100).round(1)

    # calculate the backlog by department
    backlog = total_wos - wos_completed

    # create a new dataframe for the backlog
    df_backlog = df[df['Date Closed'].isnull()]

    # calculate the time in backlog in days
    df_backlog['Time in Backlog'] = (pd.to_datetime('now') - pd.to_datetime(df_backlog['Date Open'])).dt.days

    # group the backlog data
    backlog_grouped = df_backlog.groupby(groupby_col)

    # calculate the average time in backlog for each department
    avg_time_in_backlog = backlog_grouped['Time in Backlog'].mean()

    # calculate the estimated time to clear backlog
    time_to_clear_backlog = (backlog / wos_completed * avg_time_in_backlog).round(1)
#     time_to_clear_backlog = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

    time_to_clear_backlog = time_to_clear_backlog.fillna(0)

    # create the DataFrame
    report = pd.DataFrame({'Total WOs': total_wos,
                         'WOs Completed': wos_completed,
                         'Backlog': backlog,
                         'In Progress': in_progress,
                         'New Request': new_request,
                         'Average WO Duration': avg_duration,
                         'WO Completion Rate': wo_completion_rate_percent,
                         'Estimated Time to Clear Backlog': time_to_clear_backlog})

    # sort the departments by total work orders in descending order
    
    
    report = report.sort_values(['Total WOs', 'WO Completion Rate', 'Backlog'], ascending=[False, True, False])

    report = report.rename_axis(groupby_col).reset_index()

    return report


In [7]:
# generate report by department
dept_report = generate_work_order_report(df, 'Work Category')
print(dept_report)

# generate report by campus/facility
campus_report = generate_work_order_report(df, 'Source Site')

campus_report['Site_ID'] = campus_report['Source Site'].str[-3:]
campus_report_columns = ['Site_ID', 'Source Site', 'Total WOs', 'WOs Completed', 'Backlog', 
                         'In Progress', 'New Request', 'Average WO Duration', 'WO Completion Rate',
                         'Estimated Time to Clear Backlog']
campus_report = campus_report[campus_report_columns]

print(campus_report)

# generate report by employee
# employee_report = generate_work_order_report(df, 'Source User')
# print(employee_report)

C:\Users\E178162\Anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:2199: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


                          Work Category  Total WOs  WOs Completed  Backlog  \
0               HVAC - Air Conditioning       5116           4437      679   
1                              Plumbing       4978           4643      335   
2                   Building Operations       4783           4307      476   
3                             Locksmith       4298           4262       36   
4          Life Safety Systems - Alarms       2465           2448       17   
5                     HVAC - Scheduling       2274           2253       21   
6                            Structural       2233           2140       93   
7            Integrated Pest Management       1716           1567      149   
8                            Electrical       1648           1547      101   
9                        AV/Electronics       1437           1344       93   
10                              Grounds       1302            332      970   
11              Construction Management        823            22

C:\Users\E178162\AppData\Local\Temp\ipykernel_7556\1536817795.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_backlog['Time in Backlog'] = (pd.to_datetime('now') - pd.to_datetime(df_backlog['Date Open'])).dt.days
C:\Users\E178162\Anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:2199: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(
C:\Users\E178162\AppData\Local\Temp\ipykernel_7556\1536817795.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [8]:
# calculate the total work orders in the Dept_WOs DataFrame
total_dept_wos = dept_report['Total WOs'].sum()

# calculate the total work orders received by all employees in the Employee_metrics DataFrame
# total_employee_wos = employee_report['Total WOs'].sum()

total_schools_wos = campus_report['Total WOs'].sum()

# compare the two totals
if total_dept_wos == total_schools_wos:
    print("Data validation successful: Total number of work orders match.")
else:
    print("Data validation failed: Total number of work orders do not match.")


Data validation successful: Total number of work orders match.


In [9]:
# total_dept_wos.to_excel('total_dept_wos.xlsx', index=False)
# total_employee_wos.to_excel('total_employee_wos.xlsx', index=False)


In [10]:
# Upload each dataframe into separate tables
dataframes = [work_order_report, dept_report, campus_report]
table_names = ["work_order_report", "dept_report", "campus_report"]

def clean_colname(dataframe):
    #force column names to be lower case, no spaces, no dashes
    dataframe.columns = [x.lower().replace(" ", "_") for x in dataframe.columns]
    
    #processing data
    replacements = {
       'timedelta64[ns]': 'varchar',
        'object': 'varchar',
        'float64': 'float',
        'int64': 'integer',
        'datetime64': 'timestamp',
        'int32': 'integer'
    }

    col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements)))
    
    return col_str, dataframe.columns

In [12]:
def upload_to_db(host, dbname, user, password, col_str, table_name, dataframe, cur):
    conn_string = "host={} dbname={} user={} password={}".format(host, dbname, user, password)
    conn = psycopg2.connect(conn_string)
    print('Opened database successfully')
    
    # Drop table with same name
    cur.execute("DROP TABLE IF EXISTS %s CASCADE;" % (table_name))

    # Create table
    cur.execute("CREATE TABLE %s (%s);" % (table_name, col_str))
    print('{0} was created successfully'.format(table_name)) 
    
    # Replace NaT values with NULL
    dataframe = dataframe.where(pd.notnull(dataframe), None)

    # Insert data into table
    for index, row in dataframe.iterrows():
        values = ', '.join(["NULL" if val is None else "'{}'".format(str(val).replace("'", "''")) for val in row.tolist()])
        cur.execute("INSERT INTO %s (%s) VALUES (%s);" % (table_name, ", ".join(dataframe.columns), values))
    print('Data was inserted successfully')
    
    cur.execute("GRANT SELECT ON TABLE %s TO public;" % table_name)
    
    conn.commit()
    print('Table {0} imported to db completed'.format(table_name))
    print()
    
    return 'Upload to database completed successfully'

# Create a cursor object
conn_string = "host=localhost dbname=AE_Work_Orders user=postgres password=postgres"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

# Upload each dataframe into separate tables
for i, df in enumerate(dataframes):
    col_str, dataframe_columns = clean_colname(df)
    table_name = table_names[i]
    upload_to_db("localhost", "AE_Work_Orders", "postgres", "postgres", col_str, table_name, df, cur)

# Add primary key to Table 5
cur.execute("ALTER TABLE campus_report ADD PRIMARY KEY (site_id)")
cur.execute("ALTER TABLE campus_report ALTER COLUMN site_id TYPE integer USING site_id::integer")

# Add primary key to Table 6
cur.execute("ALTER TABLE dept_report ADD PRIMARY KEY (work_category)")
cur.execute("ALTER TABLE dept_report ADD CONSTRAINT fk_work_category FOREIGN KEY (work_category) REFERENCES work_category(work_category)")

# # Add primary key to Table 7
# cur.execute("ALTER TABLE employee_report ADD PRIMARY KEY (source_user)")

# Add primary key to Table 8
cur.execute("ALTER TABLE work_order_report ADD PRIMARY KEY (work_order_id)")

# Add foreign key to Table 8
cur.execute("ALTER TABLE work_order_report ALTER COLUMN date_open TYPE date USING NULLIF(date_open, '')::date")
cur.execute("ALTER TABLE work_order_report ALTER COLUMN date_closed TYPE date USING NULLIF(date_closed, '')::date")
cur.execute("ALTER TABLE work_order_report ADD CONSTRAINT fk_work_category FOREIGN KEY (work_category) REFERENCES work_category(work_category)")
cur.execute("ALTER TABLE work_order_report ADD CONSTRAINT fk_campus_site_id FOREIGN KEY (site_id) REFERENCES campus_report(site_id)")
cur.execute("ALTER TABLE work_order_report ADD CONSTRAINT fk_site_id FOREIGN KEY (site_id) REFERENCES school_information(site_id)")
cur.execute("ALTER TABLE work_order_report ADD CONSTRAINT fk_source_user FOREIGN KEY (source_user) REFERENCES employee_report(source_user)")

conn.commit()
cur.close()


Opened database successfully
work_order_report was created successfully
Data was inserted successfully
Table work_order_report imported to db completed

Opened database successfully
dept_report was created successfully
Data was inserted successfully
Table dept_report imported to db completed

Opened database successfully
campus_report was created successfully
Data was inserted successfully
Table campus_report imported to db completed

